# BA2G Implement GibbsSampler

In [4]:
import numpy as np
import random

def formProfile(seqs):
    profiles = np.ones([4, len(seqs[0])])
    number2base = {0 : 'A', 1 : 'C', 2 : 'G', 3 : 'T'}
    for i in range(4):
        for j in range(len(seqs[0])):
            profiles[i, j] += float(sum([x[j] == number2base[i] for x in seqs]))
            profiles[i, j] = profiles[i, j]/2/len(seqs)#different than BA2D, add 1 pseudocount
    return(profiles)


def scoreMotif(seqs):
    score = 0
    profile = formProfile(seqs)
    number2base = {0 : 'A', 1 : 'C', 2 : 'G', 3 : 'T'}
    for i in range(len(seqs[0])):
        score += max(profile[:, i]) * len(seqs) * 2 - 1 #different than BA2D
    return(score)

def randomKmerfromProfile(seq, profile):
    kmers = []
    prob = []
    base2number = {'A' : 0, 'C' : 1, 'G' : 2, 'T' : 3}
    for i in range(len(seq) - profile.shape[1] + 1):
        kmers.append(seq[i : i + profile.shape[1]])
        prob.append(np.prod([profile[base2number[x], y] for x,y in zip(seq[i : i + profile.shape[1]], range(profile.shape[1]))]))
    idx = random.choices(range(len(seq) - profile.shape[1] + 1), weights = prob)
    return kmers[idx[0]]

def GibbsSampler(dna, k, t, N):
    idx = [random.choice(range(len(dna[0]) - k + 1)) for _ in range(t)]
    motifs = [dna[i][idx[i] : idx[i] + k] for i in range(t)]
    bestMotifs = motifs.copy()
    scoreBestMotifs = scoreMotif(bestMotifs)
    for j in range(N):
        seqIdx = random.choice(range(t))
        motifs.pop(seqIdx)
        profile = formProfile(motifs)
        motifs.insert(seqIdx, randomKmerfromProfile(dna[seqIdx], profile))
        if scoreMotif(motifs) > scoreBestMotifs:
            bestMotifs = motifs.copy()
            scoreBestMotifs = scoreMotif(bestMotifs)
    return bestMotifs, scoreBestMotifs

def BA2G(filename):
    with open(filename, 'r') as f:
        k, t, N = list(map(int, f.readline().rstrip().split()))
        dna = []
        for line in f:
            dna.append(line.rstrip())
        allScores = []
        allMotifs = []
        for i in range(20):
            motifs, scores = GibbsSampler(dna, k, t, N)
            allScores.append(scores)
            allMotifs.append(motifs)
        bestMotifs = allMotifs[allScores.index(max(allScores))]
        for i in bestMotifs:
            print(i)
        return bestMotifs

## Test

In [11]:
BA2G('BA2G-test.txt')

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG


['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']

In [103]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA', 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
       'TAGTACCGAGACCGAAAGAAGTATACAGGCGT', 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC', 
       'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
k = 8
t = 5
N = 100
testMotifs, testScore = GibbsSampler(dna, k, t, N)
print(testMotifs)
print(testScore)

['TCGGGGGT', 'CAAGGTGC', 'ACAGGCGT', 'TCAGGTGC', 'CCACGTGC']
30.0


In [110]:
test = scoreMotif(['AACGGCCA', 'AAGTGCCA', 'TAGTACCG', 'AAGTTTCA', 'ACGTGCAA'])
ans = scoreMotif(['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG'])
print('Test is %s and Ans is %s' % (test, ans))

Test is 31.0 and Ans is 31.0


In [71]:
bestMotifs, allScores, allMotifs = BA2G('BA2G-test.txt')
print(allScores)
print(allMotifs)

[27.0, 28.0, 29.0, 30.0, 28.0, 31.0, 28.0, 30.0, 29.0, 30.0, 28.0, 29.0, 26.0, 31.0, 28.0, 30.0, 30.0, 29.0, 30.0, 29.0, 31.0, 31.0, 29.0, 29.0, 28.0, 30.0, 28.0, 29.0, 30.0, 29.0, 29.0, 29.0, 29.0, 30.0, 28.0, 30.0, 30.0, 30.0, 30.0, 30.0]
TCTCGGGG
TGTAAGTG
TATACAGG
TTCAGGTG
TCCACGTG
[27.0, 28.0, 29.0, 30.0, 28.0, 31.0, 28.0, 30.0, 29.0, 30.0, 28.0, 29.0, 26.0, 31.0, 28.0, 30.0, 30.0, 29.0, 30.0, 29.0, 31.0, 31.0, 29.0, 29.0, 28.0, 30.0, 28.0, 29.0, 30.0, 29.0, 29.0, 29.0, 29.0, 30.0, 28.0, 30.0, 30.0, 30.0, 30.0, 30.0]
[['AAACGGCC', 'GGCGAGGT', 'CCGAAAGA', 'GGTGCACG', 'CACCAGCT'], ['CGGGGGTG', 'GCGAGGTA', 'TAGTACCG', 'GTGCACGT', 'CAGCTCCA'], ['CAGTAAAC', 'AGGTGCCA', 'AAGTATAC', 'AGGTGCAC', 'ATGTTGGC'], ['GGGGGTGT', 'CGAGGTAT', 'ACCGAAAG', 'GCACGTCG', 'TGTTGGCC'], ['GGGGTGTT', 'AGTGCCAA', 'AGTATACA', 'CGTCGGTG', 'TGTTGGCC'], ['TCTCGGGG', 'TGTAAGTG', 'TATACAGG', 'TTCAGGTG', 'TCCACGTG'], ['GTGTTCAG', 'AGGTGCCA', 'TAGTACCG', 'AGGTGCAC', 'ACGTGCAA'], ['GGGTGTTC', 'ATGTGTAA', 'TAGTACCG', '

In [12]:
BA2G('BA2G-test2.txt')

ACGTCCACCGGCGTC
AAGCGCACCGGGGTG
ACCCTTACCGGGGTG
AAGTTCCTCGGGGTG
AAGTTTTATGGGGTG
AAGTTTACCGGGTGC
AAGTTTCGAGGGGTG
CTGTTTACCGGGGTA
AAGTTGCTCGGGGTG
AAACATACCGGGGTG
AAGTTTAGGAGGGTG
AAGGAAACCGGGGTG
AAGTTTACACAGGTG
TAGTTTACCGGGGAT
CCTTTTACCGGGGTG
AAGTGAGCCGGGGTG
AAGTCGTCCGGGGTG
AAGTTTACCGGACAG
AAGTTTACCAATGTG
AAGTTTACCGTCATG


['ACGTCCACCGGCGTC',
 'AAGCGCACCGGGGTG',
 'ACCCTTACCGGGGTG',
 'AAGTTCCTCGGGGTG',
 'AAGTTTTATGGGGTG',
 'AAGTTTACCGGGTGC',
 'AAGTTTCGAGGGGTG',
 'CTGTTTACCGGGGTA',
 'AAGTTGCTCGGGGTG',
 'AAACATACCGGGGTG',
 'AAGTTTAGGAGGGTG',
 'AAGGAAACCGGGGTG',
 'AAGTTTACACAGGTG',
 'TAGTTTACCGGGGAT',
 'CCTTTTACCGGGGTG',
 'AAGTGAGCCGGGGTG',
 'AAGTCGTCCGGGGTG',
 'AAGTTTACCGGACAG',
 'AAGTTTACCAATGTG',
 'AAGTTTACCGTCATG']

## Quiz

In [13]:
BA2G('rosalind_ba2g.txt')

TGCCCGATCTCGTTG
CGCTTCGTAGAGTTG
CGCGCGGCCCAGTTG
CGCGCCCGAGAGTTG
CATACGGTAGAGTTG
CGGTAGGTAGAGTTG
CGCGCGGTAGTACTG
ACCGCGGTAGAGTTC
CGCGCGGTGACGTTG
CGCGCGGTAGAGGGA
CGCGAAATAGAGTTG
CGCGTTCTAGAGTTG
CGCGCGGTAAGCTTG
CGCGCGGTAGAAAGG
CGCGCGTGCGAGTTG
CGCGCACGAGAGTTG
GGCGCGGTAGAGTAC
CGCGCGCATGAGTTG
ATGGCGGTAGAGTTG
CGCCAAGTAGAGTTG


['TGCCCGATCTCGTTG',
 'CGCTTCGTAGAGTTG',
 'CGCGCGGCCCAGTTG',
 'CGCGCCCGAGAGTTG',
 'CATACGGTAGAGTTG',
 'CGGTAGGTAGAGTTG',
 'CGCGCGGTAGTACTG',
 'ACCGCGGTAGAGTTC',
 'CGCGCGGTGACGTTG',
 'CGCGCGGTAGAGGGA',
 'CGCGAAATAGAGTTG',
 'CGCGTTCTAGAGTTG',
 'CGCGCGGTAAGCTTG',
 'CGCGCGGTAGAAAGG',
 'CGCGCGTGCGAGTTG',
 'CGCGCACGAGAGTTG',
 'GGCGCGGTAGAGTAC',
 'CGCGCGCATGAGTTG',
 'ATGGCGGTAGAGTTG',
 'CGCCAAGTAGAGTTG']